##  ewf-ext-02-03-01 - Landsat-7 and -8 NDVI time series

Landsat-7 and -8 NDVI time series

### <a name="service">Service Definition

In [1]:
service = dict([('title', 'Landsat-7 and -8 NDVI time series'),
                ('abstract', 'Landsat-7 and -8 NDVI time series'),
                ('id', 'ewf-ext-02-03-01')])

### <a name="parameter">Parameter Definition 

**AOI**

AOI

In [2]:
areaOfInterest = dict([('id', 'areaOfInterest'),
                         ('value', 'POLYGON ((-8.908553 38.860527, -8.905850 38.863402, -8.901174 38.860026, -8.904520 38.857585, -8.908553 38.860527))'),
                         ('title', 'WKT Polygon for the Area of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

**Name of AOI**

Name of AOI

In [3]:
nameOfArea = dict([('id', 'nameOfArea'),
                     ('value', 'P001'),
                     ('title', 'Name of Area of Interest'),
                     ('abstract', 'Name of the Area of Interest'),
                     ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifier**

This is the Landsat product identifier

In [4]:
# landsat-8
#input_identifier = 'LC08_L1TP_204033_20171106_20171121_01_T1'

# landsat-7
input_identifier = 'LE72040332017334ASN00'



**Input reference**

This is the Landsat catalogue reference

In [5]:
# landsat-7
input_reference = 'https://catalog.terradue.com/landsat7/search?format=atom&uid=LE72040332017334ASN00'

# landsat-8
#input_reference = 'https://catalog.terradue.com/landsat8/search?format=atom&uid=LC08_L1TP_204033_20171106_20171121_01_T1'

**Data path**

This path defines where the data is staged-in. 

In [6]:
data_path = '/workspace/data/Landsat-7/staged-in'

data_path = '/workspace/data/Landsat-7/ftp.deimos.com.pt/Landsat_7_ETM+_C1_Level-2_3ys'

data_path = '/workspace/stage-in/teste_l8/ls7'

###  Aux folders

In [7]:
output_folder = ''

In [8]:
temp_folder = 'temp'

### <a name="workflow">Workflow

#### Import Modules

In [9]:
import os
import sys

import tarfile

from urlparse import urlparse

import shapely.wkt
import shapely.geometry
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.wkt import loads
from shapely.geometry import box
import cioppy
import lxml.etree as etree
#import gdal
#import osr

#import numpy as np
from osgeo import gdal, ogr, osr

import numpy as np
from geopandas import GeoDataFrame
import pandas as pd
import geopandas as gp

import glob

import datetime

import json



ciop = cioppy.Cioppy()

####  Auxiliary vars

In [10]:
check_results = False
export_properties_file = True

####  Auxiliary methods

In [11]:
# remove contents of a given folder
# used to clean a temporary folder
def rm_cfolder(folder):
    #folder = '/path/to/folder'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)


#
# ndvi computation
#
# Input:
# NIR - spectral reflectance measurement acquired in the red (visible) region
# red - spectral reflectance measurement acquired in the near-infrared region
# 
# Output:
# ndvi -  normalized difference vegetation index
#
def get_ndvi (nir, red):
    
    ndvi = -9999
    
    if nir >= 0 and nir <= 1 and red >= 0 and red <= 1:
        
        try:
            ndvi = ( nir - red ) / ( nir + red )
        except ZeroDivisionError:
            print("Zero Division. ndvi = -9999")
            ndvi = -9999
        except:
            print("Something else went wrong")
            return
        
    
    return ndvi


def load_img_to_matrix (img_path):
    
    dataset = gdal.Open(img_path)
    product_array = dataset.GetRasterBand(1).ReadAsArray()
    return product_array


def get_metadata(filepath):
    ds = gdal.Open(filepath)
    projection = ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    no_data_value = ds.GetRasterBand(1).GetNoDataValue()
    data_type = ds.GetRasterBand(1).DataType
    return projection, geotransform, no_data_value, data_type



def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    
    
    if mask is not None and mask is not 0:
        # TODO: check if output folder exists
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)
    



def wkt2json(wkt_str, json_file_path):

    # Convert to a shapely.geometry.polygon.Polygon object
    g1 = shapely.wkt.loads(wkt_str)

    g2 = shapely.geometry.mapping(g1)

    with open(json_file_path, 'w') as outfile:
        json.dump(g2, outfile)
    

    return json_file_path

def get_formatted_date(datetime_str):
    date = datetime.datetime.strftime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    
    first_date = datetime.datetime(year=first_date.year, month=first_date.month, day=first_date.day)
    first_date = first_date + datetime.timedelta(days=0, hours=0, minutes=0, seconds=0)
    first_date = get_formatted_date(first_date)
    
    last_date = datetime.datetime(year=last_date.year, month=last_date.month, day=last_date.day)
    last_date = last_date + datetime.timedelta(days=0, hours=23, minutes=59, seconds=59)
    last_date = get_formatted_date(last_date)
    

    
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

In [12]:
#
# crop landsat 7 given bands
#
def get_bands_geotiff_ls7_tar_gz(product_path_folder, band_names, crop=False, bbox=None):
    

    file_path = product_path_folder
    
    print(file_path)
    
    
    # open each band image
       
    for bname in band_names:
        
        try:
            print('GDAL is opening:')
        
            print('/vsitar/{0}/{1}'.format(file_path, bname))
        
            ds = gdal.Open('/vsitar/{0}/{1}'.format(file_path, bname))
        
            #ds3 = gdal.Open('/vsitar/%s/LC08_L1TP_204033_20190723_20190801_01_T1_B3.TIF' % file_path)
        
        except Exception as e:
            print e
            raise
        
    

        # crop it
    
        output = temp_folder + '/clip.tif'
    
        if crop == True:
            print 'Cropping data...'
            ulx, uly, lrx, lry = bbox[0], bbox[3], bbox[2], bbox[1]
            ds = gdal.Translate(output, ds, projWin = [ulx, uly, lrx, lry], projWinSRS = 'EPSG:4326')
            print 'data cropping : DONE'
        
        else:
            ds = gdal.Translate(output, ds)
        
    
    
        ds = None
        ds = gdal.Open(output)
        w = ds.GetRasterBand(1).XSize
        h = ds.GetRasterBand(1).YSize
        geo_transform = ds.GetGeoTransform()
        projection = ds.GetProjection()
    
    
        out_name = bname
        
        #out_name = '/'.join([temp_folder, out_name])
        out_name = os.path.join(temp_folder, out_name)
        
        print 'creating %s' %out_name
        
        data = ds.GetRasterBand(1).ReadAsArray(0, 0, w, h).astype(np.float32)
        
        if 'cloud_qa' in bname or 'pixel_qa' in bname:
            pass
        else:
            bmin = data[data!=-9999].min()
            bmax = data[data!=-9999].max()
            #Let's map the data into [0.0, 1.0]
            if bmin != bmax:
                data[data!= -9999] = (data[data!=-9999] - bmin)/(bmax - bmin)
            else:
                data[data!=-9999] = 1.0
            

    
        drv = gdal.GetDriverByName('GTiff')

        ds2 = drv.Create(out_name, w, h, 1, gdal.GDT_Float32)

        ds2.SetGeoTransform(geo_transform)
        ds2.SetProjection(projection)

        ds2.GetRasterBand(1).WriteArray(data, 0, 0)
        ds2.FlushCache()

    
        ds = None
        ds2 = None
        
        
#
# download (from T2 Catalog) and crop landsat 8 given bands
#
def get_bands_geotiff_ls8(download_url, band_indices, crop=False, bbox=None):
    
    # comeca aqui
    #url = 'https://store.terradue.com/download/landsat8/files/v1/LC08_L1TP_204033_20190723_20190801_01_T1'

    
    
    # get identifier
    l8identifier = download_url.split('/')[-1]
    
    # get band names
    band_names = []
    
    for i in band_indices:
        
        if isinstance(i, str):
            band_names.append(l8identifier + '_B{0}.TIF'.format(i))
        else:
            band_names.append(l8identifier + '_B{0}.TIF'.format(i))
        
    
    # download LS-8 Product
    retrieved = ciop.copy(download_url, temp_folder)
    
    
    
    # get product file path
    if temp_folder[-1] == '/':
        file_path = glob.glob(temp_folder + '*.tar')
    else:
        file_path = glob.glob(temp_folder + '/*.tar')

    file_path = file_path[0]
    
    print(file_path)
    
    
    
    # open each band image
       
    for bname in band_names:
        
        try:
            print('GDAL is opening:')
        
            print('/vsitar/{0}/{1}'.format(file_path, bname))
        
            ds = gdal.Open('/vsitar/{0}/{1}'.format(file_path, bname))
        
            #ds3 = gdal.Open('/vsitar/%s/LC08_L1TP_204033_20190723_20190801_01_T1_B3.TIF' % file_path)
        
        except Exception as e:
            print e
            raise
        
    

        # crop it
    
        output = temp_folder + '/clip.tif'
    
        if crop == True:
            print 'Cropping data...'
            ulx, uly, lrx, lry = bbox[0], bbox[3], bbox[2], bbox[1]
            ds = gdal.Translate(output, ds, projWin = [ulx, uly, lrx, lry], projWinSRS = 'EPSG:4326')
            print 'data cropping : DONE'
        
        else:
            ds = gdal.Translate(output, ds)
        
    
    
        ds = None
        ds = gdal.Open(output)
        w = ds.GetRasterBand(1).XSize
        h = ds.GetRasterBand(1).YSize
        geo_transform = ds.GetGeoTransform()
        projection = ds.GetProjection()
    
    
        out_name = bname
        
        #out_name = '/'.join([temp_folder, out_name])
        out_name = os.path.join(temp_folder, out_name)
        
        print 'creating %s' %out_name
        
        data = ds.GetRasterBand(1).ReadAsArray(0, 0, w, h).astype(np.float32)
        
        if '_BQA.TIF' in bname:
            pass
        else:
            bmin = data[data!=-9999].min()
            bmax = data[data!=-9999].max()
            #Let's map the data into [0.0, 1.0]
            if bmin != bmax:
                data[data!= -9999] = (data[data!=-9999] - bmin)/(bmax - bmin)
            else:
                data[data!=-9999] = 1.0
            

    
        drv = gdal.GetDriverByName('GTiff')

        ds2 = drv.Create(out_name, w, h, 1, gdal.GDT_Float32)

        ds2.SetGeoTransform(geo_transform)
        ds2.SetProjection(projection)

        ds2.GetRasterBand(1).WriteArray(data, 0, 0)
        ds2.FlushCache()

    
        ds = None
        ds2 = None

####   Auxiliary folders

In [13]:
#Create folders
#if not os.path.isdir(data_path):
#    os.mkdir(data_path)

if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

#### Define the AOI

In [14]:
aoi_wkt = loads(areaOfInterest['value'])
aoi_wkt.wkt

'POLYGON ((-8.908552999999999 38.860527, -8.905849999999999 38.863402, -8.901173999999999 38.860026, -8.90452 38.857585, -8.908552999999999 38.860527))'

In [15]:
bbox = list(aoi_wkt.bounds)
bbox

[-8.908553, 38.857585, -8.901174, 38.863402]

#### check if it is landsat 8 

In [16]:
# 'LE07' - Landsat 7
# 'LC08' - Landsat 8
lscode = 'LE07'

if 'LC08' in input_identifier:
    lscode = 'LC08'
    print('Landsat 8')
else:
    print('Landsat 7')

Landsat 7


#### opens tar file

In [17]:
targz_file = glob.glob(data_path + '/*.tar')
ls_file_path = targz_file[0]

t = tarfile.open(ls_file_path, 'r')
print t.getnames()

#ls_file_path


['LE07_L1TP_204033_20171130_20171226_01_T1_ANG.txt', 'LE07_L1TP_204033_20171130_20171226_01_T1_MTL.txt', 'LE07_L1TP_204033_20171130_20171226_01_T1_pixel_qa.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_radsat_qa.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_band1.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_band2.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_band3.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_band4.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_band5.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_band7.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_atmos_opacity.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_cloud_qa.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1.xml']


In [18]:
data_path

'/workspace/stage-in/teste_l8/ls7'

In [19]:
band_names = []

band_1_name = ''
band_2_name = ''
band_3_name = ''

for n in t.getnames():
    if 'band4' in n:
        band_names.append(n)
        band_1_name = n
        
    if 'band3' in n:
        band_names.append(n)
        band_2_name = n
        
    #if 'pixel_qa' in n:
    #    band_names.append(n)
        
    if 'cloud_qa' in n:
        band_names.append(n)
        band_3_name = n
        
band_1_name = os.path.join(temp_folder, band_1_name)
band_2_name = os.path.join(temp_folder, band_2_name)
band_3_name = os.path.join(temp_folder, band_3_name) # QA band
        
print(band_names)

print(band_1_name)
print(band_2_name)
print(band_3_name)

get_bands_geotiff_ls7_tar_gz(ls_file_path, band_names, crop=True, bbox=bbox)

['LE07_L1TP_204033_20171130_20171226_01_T1_sr_band3.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_band4.tif', 'LE07_L1TP_204033_20171130_20171226_01_T1_sr_cloud_qa.tif']
temp/LE07_L1TP_204033_20171130_20171226_01_T1_sr_band4.tif
temp/LE07_L1TP_204033_20171130_20171226_01_T1_sr_band3.tif
temp/LE07_L1TP_204033_20171130_20171226_01_T1_sr_cloud_qa.tif
/workspace/stage-in/teste_l8/ls7/LE072040332017113001T1-SC20190921113409.tar
GDAL is opening:
/vsitar//workspace/stage-in/teste_l8/ls7/LE072040332017113001T1-SC20190921113409.tar/LE07_L1TP_204033_20171130_20171226_01_T1_sr_band3.tif
Cropping data...
data cropping : DONE
creating temp/LE07_L1TP_204033_20171130_20171226_01_T1_sr_band3.tif
GDAL is opening:
/vsitar//workspace/stage-in/teste_l8/ls7/LE072040332017113001T1-SC20190921113409.tar/LE07_L1TP_204033_20171130_20171226_01_T1_sr_band4.tif
Cropping data...
data cropping : DONE
creating temp/LE07_L1TP_204033_20171130_20171226_01_T1_sr_band4.tif
GDAL is opening:
/vsitar//workspace/stage-in

In [20]:
#
# Computes ndvi image
# and saves it in geotif
#
def get_cloud_mask(band_qa):
    
    print('GENERATING MASK from: {0}'.format(band_qa))
    
    mat_band_qa = load_img_to_matrix(band_qa)
    
    cloud_mask = np.full_like(mat_band_qa, 0)
    
    # 1st row - clouds
    # 2nd row - Cloud shadow - high
    # codes from here: https://www.usgs.gov/media/images/landsat-8-quality-assessment-band-attributes-and-possible-values
    # and from here: LSDS-1370_L4-7_SurfaceReflectance-LEDAPS_ProductGuide-v2.pdf
    if 'LC08_' in band_qa:
        pixel_values_to_mask = [2800, 2804, 2808, 2812, 6896, 6900, 6904, 6908,
                                2976, 2980, 2984, 2988, 3008, 3012, 3016, 3020, 7072, 7076, 7080, 7084, 7104, 7108, 7112, 7116]
    else:
        #pixel_values_to_mask = [96, 112, 160, 176, 224,
        #                        72, 136]
        
        pixel_values_to_mask = [2, 34,
                                4, 12, 20, 36, 52]
    
    for pv in pixel_values_to_mask:
        cloud_mask[mat_band_qa == pv] = -9999
    
    return cloud_mask



#
# Computes ndvi image
# and saves it in geotif
#
def normalized_difference(band_1, band_2, output_name, mask_clouds = False, band_3 = None):

    #metadata_name = '{0}.xml'.format(output_name)
    
    #otb_app = otbApplication.Registry.CreateApplication('BandMath')
    
    #otb_app.SetParameterStringList('il', [band_1, band_2])

    #otb_app.SetParameterString('out', '{0}.TIF'.format(output_name))
    
    #otb_app.SetParameterString('exp', 'im1b1 >= 0 && im1b1 <= 1 && im2b1 >= 0 && im2b1 <= 1 ? ( im1b1 - im2b1 ) / ( im1b1 + im2b1 ) : -9999 ')

    #otb_app.ExecuteAndWriteOutput()
    
    
    mat_band_1 = load_img_to_matrix(band_1)
    mat_band_2 = load_img_to_matrix(band_2)
    
    
    ndvi_calculator = np.vectorize(get_ndvi)
    
    mat_ndvi = ndvi_calculator(mat_band_1, mat_band_2)
    
    if mask_clouds: # mask clouds and cloud shadows
        
        cloud_mask = get_cloud_mask(band_3)
        
        mat_ndvi[cloud_mask == -9999] = -9999
        
    
    projection, geotransform, no_data_value, data_type = get_metadata(band_1)
    
    no_data_value = -9999

    write_output_image(output_name + '.tif', mat_ndvi, 'GTiff', data_type, None, projection, geotransform, no_data_value)
    
    
    #Set the actual wkt for NDVI 
    #metadata['wkt'] = get_wkt('{0}.TIF'.format(output_name))
    
    #metadata['identifier'] = output_name
    
    return True



output_name = '{0}{1}_{2}'.format(band_1_name.split('/')[-1].split('.')[0].split('band')[0], nameOfArea['value'],'NDVI')

print(output_name)

ciop.log("INFO", "Computing NDVI")

normalized_difference(band_1_name,
                      band_2_name,
                      output_name,
                      mask_clouds = True,
                      band_3 = band_3_name)

LE07_L1TP_204033_20171130_20171226_01_T1_sr_P001_NDVI
GENERATING MASK from: temp/LE07_L1TP_204033_20171130_20171226_01_T1_sr_cloud_qa.tif


reporter:status:2019-11-02T12:37:25.273141 [INFO   ] [user process] Computing NDVI
2019-11-02T12:37:25.273141 [INFO   ] [user process] Computing NDVI


True

In [21]:
if check_results:
    
    #lscode = 'LC07'
    
    import matplotlib
    import matplotlib.pyplot as plt
    from PIL import Image
    %matplotlib inline
    
    ds = gdal.Open(band_1_name)
    band4 = ds.GetRasterBand(1).ReadAsArray()
    ds=None

    ds = gdal.Open(band_2_name)
    band3 = ds.GetRasterBand(1).ReadAsArray()
    ds=None


    for b in [band3, band4]:

        bmin = b[b!=-9999].min()
        bmax =  b[b!=-9999].max()
        
        if bmin != bmax:    
            b[b!=-9999] = (b[b!=-9999] - bmin)/(bmax - bmin) * 255
  
        b[b==-9999]=np.nan

    fig = plt.figure(figsize=(20,20))

    a = fig.add_subplot(1,2,1)
    imgplot = plt.imshow(band3.astype(np.uint8),cmap='jet')
    a.set_title('LS7 Band 3')
    if lscode == 'LC08':
        a.set_title('LS8 Band 4')
    

    a = fig.add_subplot(1,2,2)
    imgplot = plt.imshow(band4.astype(np.uint8),cmap='jet')
    a.set_title('LS7 Band 4')
    if lscode == 'LC08':
        a.set_title('LS8 Band 5')
        

    plt.tight_layout()
    fig = plt.gcf()
    plt.show()

    fig.clf()
    plt.close()

#### clip according to AOI

In [22]:
input_raster = '{0}.tif'.format(output_name)
output_raster = '{0}.tif'.format(output_name)
parcel_geojson = wkt2json(aoi_wkt.wkt, os.path.join(temp_folder, 'aoi.json'))

ds = gdal.Warp(output_raster,
              input_raster,
              format = 'GTiff',
              cutlineDSName = parcel_geojson,
              dstNodata = -9999.0)
ds = None

#### Check results

In [23]:
if check_results:
    import matplotlib
    import matplotlib.pyplot as plt
    from PIL import Image
    %matplotlib inline


In [24]:
if check_results:
    
    ds = gdal.Open(band_1_name)
    band4 = ds.GetRasterBand(1).ReadAsArray()
    ds=None

    ds = gdal.Open(band_2_name)
    band3 = ds.GetRasterBand(1).ReadAsArray()
    ds=None

    ds = gdal.Open('{0}.tif'.format(output_name))
    ndvi = ds.GetRasterBand(1).ReadAsArray()
    ds=None

    for b in [band3, band4, ndvi]:

        bmin = b[b!=-9999].min()
        bmax =  b[b!=-9999].max()
        
        if bmin != bmax:    
            b[b!=-9999] = (b[b!=-9999] - bmin)/(bmax - bmin) * 255
  
        b[b==-9999]=np.nan

    fig = plt.figure(figsize=(20,20))

    a = fig.add_subplot(1,3,1)
    imgplot = plt.imshow(band3.astype(np.uint8),cmap='gray')
    a.set_title('LS7 Band 3')
    if lscode == 'LC08':
        a.set_title('LS8 Band 4')
    

    a = fig.add_subplot(1,3,2)
    imgplot = plt.imshow(band4.astype(np.uint8),cmap='gray')
    a.set_title('LS7 Band 4')
    if lscode == 'LC08':
        a.set_title('LS8 Band 5')
        

    a = fig.add_subplot(1,3,3)
    imgplot = plt.imshow(ndvi.astype(np.uint8),cmap='jet')
    a.set_title('NDVI')

    plt.tight_layout()
    fig = plt.gcf()
    plt.show()

    fig.clf()
    plt.close()


#### Export properties file

In [25]:
if export_properties_file:
    
    write_properties_file('{0}.tif'.format(output_name), datetime.datetime.strptime(output_raster[17:17+8], '%Y%m%d'), datetime.datetime.strptime(output_raster[17:17+8], '%Y%m%d'), areaOfInterest['value'])

#### Remove temporay files and folders

In [26]:
ciop.log("INFO", "Removing aux data")

rm_cfolder(temp_folder)

os.rmdir(temp_folder)

reporter:status:2019-11-02T12:37:37.615786 [INFO   ] [user process] Removing aux data
2019-11-02T12:37:37.615786 [INFO   ] [user process] Removing aux data
